In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load the dataset from Google Drive into a pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PhD_Thesis_Experiments/GitHub_ToChair/FilteredDataSetForExp_002.csv')

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import time
from openai import AzureOpenAI
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from datetime import datetime

In [ ]:
# # ==================== Configuration ====================
# AZURE_OPENAI_ENDPOINT = "https://datascienceagenticaiwork.cognitiveservices.azure.com/"
# AZURE_OPENAI_KEY = "9BJoJeBtsvPKHy0P46u8SqHDkdsOTSzGAgE2UPAdqLocFQ5s6yTHJQQJ99BIACHYHv6XJ3w3AAAAACOGMP8G"
# AZURE_OPENAI_API_VERSION = "2024-12-01-preview"  # Use appropriate API version
# BASE_MODEL = "gpt-4.1"  # Base model for fine-tuning


In [ ]:
# ==================== Configuration ====================
AZURE_OPENAI_ENDPOINT = ""
AZURE_OPENAI_KEY = ""
AZURE_OPENAI_API_VERSION = "2024-12-01-preview"  # Use appropriate API version
BASE_MODEL = "gpt-4o-mini"  # Base model for fine-tuning
deployment = "gpt-4o-mini-tobefinetune"


In [ ]:
endpoint = "https://datascienceagenticaiwork.cognitiveservices.azure.com/"
model_name = "gpt-4o-mini"
deployment = "gpt-4o-mini-tobefinetune"

subscription_key = ""
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_completion_tokens=13107,
    temperature=1.0,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    model=deployment
)

print(response.choices[0].message.content)

Paris is a city brimming with history, art, architecture, and culture. Here are some must-see attractions and experiences while you’re there:

1. **Eiffel Tower**: No trip to Paris is complete without a visit to this iconic landmark. You can take an elevator or climb the stairs to enjoy spectacular views of the city.

2. **Louvre Museum**: Home to thousands of works of art, including the Mona Lisa and the Venus de Milo, the Louvre is a must for art lovers. Consider pre-booking tickets to avoid long lines.

3. **Notre-Dame Cathedral**: Though it is undergoing restoration after the fire in 2019, the exterior is still a marvel, and you can explore the Île de la Cité where it is located.

4. **Sacré-Cœur Basilica**: Located on the highest point in the city at Montmartre, this beautiful basilica offers amazing panoramic views of Paris.

5. **Champs-Élysées and Arc de Triomphe**: Take a leisurely stroll down this famous avenue, packed with shops and cafes, and visit the Arc de Triomphe, a mo

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Function to split hierarchical labels into product and sub-product
def split_hierarchical_label(label):
    if '::' in label:
        return label.split('::')
    else:
        return [label, 'None'] # Handle cases with no sub-product

# Function to calculate hierarchical metrics
def hierarchical_metrics(y_true, y_pred):
    product_true = [split_hierarchical_label(label)[0] for label in y_true]
    sub_product_true = [split_hierarchical_label(label)[1] for label in y_true]
    product_pred = [split_hierarchical_label(label)[0] for label in y_pred]
    sub_product_pred = [split_hierarchical_label(label)[1] for label in y_pred]

    # Calculate metrics at the product level
    product_precision = precision_score(product_true, product_pred, average='weighted', zero_division=0)
    product_recall = recall_score(product_true, product_pred, average='weighted', zero_division=0)
    product_f1 = f1_score(product_true, product_pred, average='weighted', zero_division=0)

    # Calculate metrics at the sub-product level (only for non-None sub-products)
    # We need to filter for cases where both true and predicted sub-products are not 'None'
    valid_sub_product_true = [sub for i, sub in enumerate(sub_product_true) if sub != 'None' and sub_product_pred[i] != 'None']
    valid_sub_product_pred = [sub for i, sub in enumerate(sub_product_pred) if sub != 'None' and sub_product_true[i] != 'None']


    sub_product_precision = precision_score(valid_sub_product_true, valid_sub_product_pred, average='weighted', zero_division=0) if valid_sub_product_true else 0
    sub_product_recall = recall_score(valid_sub_product_true, valid_sub_product_pred, average='weighted', zero_division=0) if valid_sub_product_true else 0
    sub_product_f1 = f1_score(valid_sub_product_true, valid_sub_product_pred, average='weighted', zero_division=0) if valid_sub_product_true else 0


    # A simple way to combine scores (can be weighted based on importance)
    # Here, we'll just average them
    hierarchical_precision = (product_precision + sub_product_precision) / 2
    hierarchical_recall = (product_recall + sub_product_recall) / 2
    hierarchical_f1 = (product_f1 + sub_product_f1) / 2

    return {
        'product_precision': product_precision,
        'product_recall': product_recall,
        'product_f1': product_f1,
        'sub_product_precision': sub_product_precision,
        'sub_product_recall': sub_product_recall,
        'sub_product_f1': sub_product_f1,
        'hierarchical_precision': hierarchical_precision,
        'hierarchical_recall': hierarchical_recall,
        'hierarchical_f1': hierarchical_f1
    }


In [ ]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import os

# ------------------------------
# 1️⃣ Load or use your dataframe
# ------------------------------
# If already in memory: df = your DataFrame
# Example if loading from CSV:
# df = pd.read_csv("complaints.csv")

# Ensure required columns exist
assert {"consumer_complaint_narrative", "hierarchical_label"}.issubset(df.columns), \
    "DataFrame must contain 'consumer_complaint_narrative' and 'hierarchical_label' columns."

# Drop rows with missing values
df = df.dropna(subset=["consumer_complaint_narrative", "hierarchical_label"])

# ------------------------------
# 2️⃣ Split into train & validation
# ------------------------------
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df["hierarchical_label"])

print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")

# ------------------------------
# 3️⃣ Helper function to save JSONL
# ------------------------------
def save_jsonl(dataframe, output_path):
    """Save dataframe to Azure OpenAI fine-tuning JSONL format."""
    with open(output_path, "w", encoding="utf-8") as f:
        for _, row in dataframe.iterrows():
            complaint_text = str(row["consumer_complaint_narrative"]).replace("\n", " ").replace("\r", " ").strip()
            label = str(row["hierarchical_label"]).strip()

            record = {
                "messages": [
                    {
                        "role": "system",
                        "content": "You are a helpful assistant that classifies consumer financial complaints into categories."
                    },
                    {
                        "role": "user",
                        "content": complaint_text
                    },
                    {
                        "role": "assistant",
                        "content": label
                    }
                ]
            }

            f.write(json.dumps(record, ensure_ascii=False) + "\n")

    print(f"✅ File saved: {output_path} ({len(dataframe)} records)")

# ------------------------------
# 4️⃣ Create output directory & save
# ------------------------------
os.makedirs("fine_tune_data", exist_ok=True)

save_jsonl(train_df, "fine_tune_data/train_data.jsonl")
save_jsonl(val_df, "fine_tune_data/val_data.jsonl")

print("🎯 Training and validation JSONL files are ready for Azure OpenAI fine-tuning.")


Training samples: 97200
Validation samples: 10800
✅ File saved: fine_tune_data/train_data.jsonl (97200 records)
✅ File saved: fine_tune_data/val_data.jsonl (10800 records)
🎯 Training and validation JSONL files are ready for Azure OpenAI fine-tuning.


In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    api_version=AZURE_OPENAI_API_VERSION
)

In [ ]:
# Upload files directly from Python
with open("fine_tune_data/train_data.jsonl", "rb") as f:
    train_file = client.files.create(file=f, purpose="fine-tune")

with open("fine_tune_data/val_data.jsonl", "rb") as f:
    val_file = client.files.create(file=f, purpose="fine-tune")

print("Train file ID:", train_file.id)
print("Validation file ID:", val_file.id)

Train file ID: file-d8cff2cea0464237909e6c4b4b390c9d
Validation file ID: file-e6d3f76212a14ab5bded087feb0b0b1d


### Full Supervised Fine‑Tuning (SFT) on Azure OpenAI GPT Models

In [ ]:
# ✅ Continue using the same client and configuration from before
fine_tune = client.fine_tuning.jobs.create(
    model = "gpt-4o-2024-08-06",   # BASE_MODEL,  # "gpt-4.1" or "gpt-4.1-mini"
    training_file="file-ac2162910b0442979cf67e1323c8cd0a",
    validation_file="file-77ec932e57444752af3b0eae4524f0e2",
    hyperparameters={
        "n_epochs": 3  # you can tune this; 2–4 is typical
    },
    suffix="cfpb-complaint-classifier"
)

print("✅ Fine-tuning job started!")
print("Job ID:", fine_tune.id)


✅ Fine-tuning job started!
Job ID: ftjob-99dbcb3dc9204bf6acd5d84748e92a8e


In [ ]:
#### Following code is just for testing to list down the name of the models which Azure OpenAI supports for Finetunning #######

# # pip install --upgrade openai
# from openai import AzureOpenAI

# # ---- Azure config (replace these) ----
# AZURE_OPENAI_ENDPOINT = "https://datascienceagenticaiwork.cognitiveservices.azure.com/"
# AZURE_OPENAI_KEY = "9BJoJeBtsvPKHy0P46u8SqHDkdsOTSzGAgE2UPAdqLocFQ5s6yTHJQQJ99BIACHYHv6XJ3w3AAAAACOGMP8G"
# API_VERSION = "2024-12-01-preview"  # recent version with Models List, FT

# client = AzureOpenAI(
#     api_key=AZURE_OPENAI_KEY,
#     api_version=API_VERSION,
#     azure_endpoint=AZURE_OPENAI_ENDPOINT,
# )

# # 1) Discover fine‑tunable models for THIS resource+region
# models = client.models.list()
# fine_tunable_ids = []
# for m in models.data:
#     # m.capabilities is a dict-like object on Azure; defensive access:
#     caps = getattr(m, "capabilities", None) or {}
#     if caps.get("fine_tune", False):
#         fine_tunable_ids.append(m.id)

# print("Fine‑tunable models available to this resource/region:")
# for mid in fine_tunable_ids:
#     print("  -", mid)

# # 2) Prefer a gpt‑4o‑mini snapshot if present (e.g., "gpt-4o-mini-2024-07-18")
# base_model_id = next((mid for mid in fine_tunable_ids if "gpt-4o-mini" in mid), None)
# if not base_model_id:
#     raise RuntimeError(
#         "No fine‑tunable gpt‑4o‑mini snapshot is enabled for this resource/region.\n"
#         "Pick any model from the printed list (or switch to a region where gpt‑4o‑mini is trainable)."
#     )

# print("Using base model:", base_model_id)

# # 3) Ensure the training/validation files exist in THIS resource and were uploaded with purpose='fine-tune'
# training_file_id   = "file-ac2162910b0442979cf67e1323c8cd0a"
# validation_file_id = "file-77ec932e57444752af3b0eae4524f0e2"

# # 4) Create the fine‑tuning job (no per-call api-version needed; client handles it)
# ft_job = client.fine_tuning.jobs.create(
#     model=base_model_id,
#     training_file=training_file_id,
#     validation_file=validation_file_id,
#     hyperparameters={"n_epochs": 3}
# )

# print("FT Job ID:", ft_job.id)



In [ ]:
status = client.fine_tuning.jobs.retrieve("ftjob-99dbcb3dc9204bf6acd5d84748e92a8e")
print("Status:", status.status)

Status: succeeded


### Get the fine‑tuned model ID

In [ ]:
from openai import AzureOpenAI

job_id = "ftjob-99dbcb3dc9204bf6acd5d84748e92a8e"
job = client.fine_tuning.jobs.retrieve(job_id)
print("Status:", job.status)
print("Fine-tuned model ID:", job.fine_tuned_model)

FT_MODEL_ID = job.fine_tuned_model  # e.g., 'ft:gpt-4o-2024-08-06:complaints-v1-2025-11-05'

Status: succeeded
Fine-tuned model ID: gpt-4o-2024-08-06.ft-99dbcb3dc9204bf6acd5d84748e92a8e-cfpb-complaint-classifier


### Using Azure Portal deployed the Fine-tuned model ID: gpt-4o-2024-08-06.ft-36293937da9a4d58bc21d14dcf13f813-cfpb-complaint-classifier with name:  gpt-4o-2024-08-06-custom-multiclass-classifier

In [ ]:
import pandas as pd
import numpy as np
import os
from openai import AzureOpenAI
import random
import asyncio
import aiohttp
from tqdm import tqdm
from openai import AsyncAzureOpenAI

In [ ]:
#DEPLOYMENT_NAME = "gpt-4o-2024-08-06-custom-multiclass-classifier"
                  # gpt-4o-2024-08-06-cfpb-complaint-classifier

In [ ]:
# ==========================================================
# 2️⃣ Dataset Setup
# ==========================================================
# Example: df has ['consumer_complaint_narrative', 'hierarchical_label']
# df = pd.read_csv('cfpb_complaints.csv')
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['hierarchical_label'])
unique_labels = df['hierarchical_label'].unique().tolist()

In [ ]:
AZURE_API_KEY = "9BJoJeBtsvPKHy0P46u8SqHDkdsOTSzGAgE2UPAdqLocFQ5s6yTHJQQJ99BIACHYHv6XJ3w3AAAAACOGMP8G"
AZURE_ENDPOINT = "https://datascienceagenticaiwork.cognitiveservices.azure.com/"
API_VERSION = "2024-12-01-preview"
MODEL_NAME = "gpt-4o"
DEPLOYMENT_NAME = "gpt-4o-2024-08-06-custom-multiclass-classifier"

# -----------------------------
# INITIALIZE CLIENT
# -----------------------------
# Use AsyncAzureOpenAI for asynchronous operations
acient = AsyncAzureOpenAI(
    api_key=AZURE_API_KEY,
    api_version=API_VERSION,
    azure_endpoint=AZURE_ENDPOINT
)

# --- Set your Azure OpenAI credentials and endpoint ---
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_API_KEY  # <-- Put your key (or store securely)
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_ENDPOINT # <-- Your endpoint

# Create the Azure OpenAI client (use the API version supported by your resource)
client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version = API_VERSION,
    azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
)

# Use your **deployment name** (not the base model name)
# deployment_name = MODEL_NAME  # e.g., "gpt4o-mini-prod"

In [ ]:
# ==========================================================
# 4️⃣ Async GPT-4.1 Zero-Shot Classifier
# ==========================================================
import asyncio
import random

async def classify_with_gpt40(session, text, label_list, max_retries=3):
    prompt = f"""
    You are a financial complaint classifier.
    Given the following consumer complaint, classify it into one of these hierarchical categories:
    {', '.join(label_list)}

    Complaint:
    "{text}"

    Respond with exactly one label from the list above.
    """

    url = f"{AZURE_ENDPOINT}openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version={API_VERSION}"
    headers = {"Content-Type": "application/json", "api-key": AZURE_API_KEY}
    payload = {
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.0,
        "max_tokens": 50
    }

    for attempt in range(max_retries):
        try:
            async with session.post(url, headers=headers, json=payload, timeout=60) as resp:
                if resp.status == 200:
                    data = await resp.json()
                    return data["choices"][0]["message"]["content"].strip()

                # Handle 429 (Rate Limit)
                elif resp.status == 429:
                    await asyncio.sleep(1 + random.random())  # small random delay
                    continue  # retry silently

                # Handle transient server errors quietly
                elif 500 <= resp.status < 600:
                    await asyncio.sleep(1 + random.random())
                    continue  # retry silently

                # For other non-critical errors: skip quietly
                else:
                    return ""

        except Exception:
            # Suppress exceptions silently, retry
            await asyncio.sleep(0.5)
            continue

    # Return empty label if all retries fail
    return ""


In [ ]:
# ==========================================================
# 5️⃣ Parallel Inference
# ==========================================================
async def run_parallel_inference(test_data, label_list, max_concurrent=5):
    y_true, y_pred = [], []
    semaphore = asyncio.Semaphore(max_concurrent)

    async with aiohttp.ClientSession() as session:
        tasks = []

        async def process_row(row):
            async with semaphore:
                predicted = await classify_with_gpt40(session, row['consumer_complaint_narrative'], label_list)
                return row['hierarchical_label'], predicted

        for _, row in test_data.iterrows():
            tasks.append(process_row(row))

        for f in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Classifying", leave=False):
            true_label, pred_label = await f
            y_true.append(true_label)
            y_pred.append(pred_label)

    return y_true, y_pred


In [ ]:
# ==========================================================
# 6️⃣ Main Execution
# ==========================================================
# y_true, y_pred = asyncio.run(run_parallel_inference(test_df, unique_labels, max_concurrent=5))
import nest_asyncio
import asyncio

nest_asyncio.apply()  # allows reusing event loop inside notebook

# Run async inference
y_true, y_pred = await run_parallel_inference(test_df, unique_labels, max_concurrent=5)



In [ ]:
y_pred

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',


In [ ]:
# ==========================================================
# 7️⃣ Evaluate Results
# ==========================================================
metrics = hierarchical_metrics(y_true, y_pred)
for key, value in metrics.items():
    print(f"{key.replace('_', ' ').title()}: {value:.4f}")

Product Precision: 0.0000
Product Recall: 0.0000
Product F1: 0.0000
Sub Product Precision: 0.0000
Sub Product Recall: 0.0000
Sub Product F1: 0.0000
Hierarchical Precision: 0.0000
Hierarchical Recall: 0.0000
Hierarchical F1: 0.0000


### **Model Performance Comparison**

---

#### **GPT-4 Models**
| **Metric**                 | **GPT-4 Zero-Shot** | **GPT-4 Fine-Tuned** |
| -------------------------- | :------------------: | :-------------------: |
| **Product Precision**      |       0.7751        |        0.7702        |
| **Product Recall**         |       0.5369        |        0.6328        |
| **Product F1**             |       0.6016        |        0.6798        |
| **Sub Product Precision**  |       0.6739        |        0.6596        |
| **Sub Product Recall**     |       0.5061        |        0.5737        |
| **Sub Product F1**         |       0.5022        |        0.5621        |
| **Hierarchical Precision** |       0.7245        |        0.7149        |
| **Hierarchical Recall**    |       0.5215        |        0.6033        |
| **Hierarchical F1**        |       0.5519        |        0.6210        |

---

#### **Classical ML & Neural Network Models**
| **Metric**                 | **Logistic Regression** | **Linear SVM** | **Decision Tree** | **Random Forest** | **LSTM** | **Bi-LSTM** | **CNN-1D** |
| -------------------------- | :----------------------: | :------------: | :---------------: | :---------------: | :------: | :---------: | :--------: |
| **Product Precision**      |         0.8799          |     0.8762     |      0.8365       |      0.8635       | 0.8798   |   0.8867    |  0.8457    |
| **Product Recall**         |         0.8859          |     0.8858     |      0.8436       |      0.8618       | 0.8901   |   0.8908    |  0.8597    |
| **Product F1**             |         0.8704          |     0.8716     |      0.8395       |      0.8379       | 0.8771   |   0.8772    |  0.8299    |
| **Sub Product Precision**  |         0.8524          |     0.8467     |      0.8098       |      0.8249       | 0.8365   |   0.8454    |  0.7911    |
| **Sub Product Recall**     |         0.8704          |     0.8692     |      0.8201       |      0.8533       | 0.8661   |   0.8681    |  0.8490    |
| **Sub Product F1**         |         0.8500          |     0.8524     |      0.8145       |      0.8173       | 0.8467   |   0.8500    |  0.8091    |
| **Hierarchical Precision** |         0.8661          |     0.8614     |      0.8231       |      0.8442       | 0.8581   |   0.8661    |  0.8184    |
| **Hierarchical Recall**    |         0.8781          |     0.8775     |      0.8319       |      0.8576       | 0.8787   |   0.8799    |  0.8544    |
| **Hierarchical F1**        |         0.8602          |     0.8620     |      0.8270       |      0.8276       | 0.8619   |   0.8636    |  0.8195    |

---

#### **Transformer Models**
| **Metric**                 | **BERT** | **FinBERT** |
| -------------------------- | :------: | :---------: |
| **Product Precision**      |  0.9897  |    0.9901   |
| **Product Recall**         |  0.9897  |    0.9900   |
| **Product F1**             |  0.9897  |    0.9900   |
| **Sub Product Precision**  |  0.9835  |    0.9836   |
| **Sub Product Recall**     |  0.9834  |    0.9834   |
| **Sub Product F1**         |  0.9834  |    0.9834   |
| **Hierarchical Precision** |  0.9866  |    0.9868   |
| **Hierarchical Recall**    |  0.9866  |    0.9867   |


### **Model Performance Comparison**

---

#### **LLM Models**
| **Metric**                 | **GPT-4 Zero-Shot** | **GPT-4 Fine-Tuned** | **Mistral Zero-Shot** | **Mistral Fine-Tuned (LoRA)** |
| -------------------------- | :------------------: | :-------------------: | :-------------------: | :---------------------------: |
| **Product Precision**      |       0.7751        |        0.7702        |        0.6102        |            0.9339            |
| **Product Recall**         |       0.5369        |        0.6328        |        0.5156        |            0.7262            |
| **Product F1**             |       0.6016        |        0.6798        |        0.5326        |            0.8125            |
| **Sub Product Precision**  |       0.6739        |        0.6596        |        0.5945        |            0.8477            |
| **Sub Product Recall**     |       0.5061        |        0.5737        |        0.2526        |            0.1154            |
| **Sub Product F1**         |       0.5022        |        0.5621        |        0.2887        |            0.1404            |
| **Hierarchical Precision** |       0.7245        |        0.7149        |        0.6023        |            0.8908            |
| **Hierarchical Recall**    |       0.5215        |        0.6033        |        0.3841        |            0.4208            |
| **Hierarchical F1**        |       0.5519        |        0.6210        |        0.4107        |            0.4764            |

---

#### **Classical ML & Neural Network Models**
| **Metric**                 | **Logistic Regression** | **Linear SVM** | **Decision Tree** | **Random Forest** | **LSTM** | **Bi-LSTM** | **CNN-1D** |
| -------------------------- | :----------------------: | :------------: | :---------------: | :---------------: | :------: | :---------: | :--------: |
| **Product Precision**      |         0.8799          |     0.8762     |      0.8365       |      0.8635       | 0.8798   |   0.8867    |  0.8457    |
| **Product Recall**         |         0.8859          |     0.8858     |      0.8436       |      0.8618       | 0.8901   |   0.8908    |  0.8597    |
| **Product F1**             |         0.8704          |     0.8716     |      0.8395       |      0.8379       | 0.8771   |   0.8772    |  0.8299    |
| **Sub Product Precision**  |         0.8524          |     0.8467     |      0.8098       |      0.8249       | 0.8365   |   0.8454    |  0.7911    |
| **Sub Product Recall**     |         0.8704          |     0.8692     |      0.8201       |      0.8533       | 0.8661   |   0.8681    |  0.8490    |
| **Sub Product F1**         |         0.8500          |     0.8524     |      0.8145       |      0.8173       | 0.8467   |   0.8500    |  0.8091    |
| **Hierarchical Precision** |         0.8661          |     0.8614     |      0.8231       |      0.8442       | 0.8581   |   0.8661    |  0.8184    |
| **Hierarchical Recall**    |         0.8781          |     0.8775     |      0.8319       |      0.8576       | 0.8787   |   0.8799    |  0.8544    |
| **Hierarchical F1**        |         0.8602          |     0.8620     |      0.8270       |      0.8276       | 0.8619   |   0.8636    |  0.8195    |

---

#### **Transformer Models**
| **Metric**                 | **BERT** | **FinBERT** |
| -------------------------- | :------: | :---------: |
| **Product Precision**      |  0.9897  |    0.9901   |
| **Product Recall**         |  0.9897  |    0.9900   |
| **Product F1**             |  0.9897  |    0.9900   |
| **Sub Product Precision**  |  0.9835  |    0.9836   |
| **Sub Product Recall**     |  0.9834  |    0.9834   |
| **Sub Product F1**         |  0.9834  |    0.9834   |
| **Hierarchical Precision** |  0.9866  |    0.9868   |
| **Hierarchical Recall**    |  0.9866  |    0.9867   |
